In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
driver_ids = pd.read_csv('driver_ids.csv')
ride_timestamps = pd.read_csv('ride_timestamps.csv')
ride_ids = pd.read_csv('ride_ids.csv')

In [4]:
driver_ids.sample(2)

,driver_id,driver_onboard_date
341,5bd37993cfff08be940348a466912238,2016-04-13 00:00:00
548,8dfea9e24286290d2f59f5b51f00b6c2,2016-04-01 00:00:00


In [5]:
print('Shape of the data:', driver_ids.shape)
print('unique of driver id:', len(driver_ids['driver_id'].unique()))
print('Minimum date:', driver_ids['driver_onboard_date'].min())
print('Maximum date:', driver_ids['driver_onboard_date'].max())

Shape of the data: (937, 2)
unique of driver id: 937
Minimum date: 2016-03-28 00:00:00
Maximum date: 2016-05-15 00:00:00


In [16]:
ride_timestamps.sample(2)

,ride_id,event,timestamp
933806,f64e9f5c5f6201ed8844d747974bd13d,accepted_at,2016-06-10 10:31:33
792529,d1381cfb8d2e4823ef032bd831737fda,dropped_off_at,2016-04-14 08:39:45


In [7]:
print('Shape of data:', ride_timestamps.shape)
print('unique of ride id:', len(ride_timestamps['ride_id'].unique()))
ride_timestamps['event'].value_counts()

Shape of data: (970405, 3)
unique of ride id: 194081


picked_up_at      194081
arrived_at        194081
requested_at      194081
dropped_off_at    194081
accepted_at       194081
Name: event, dtype: int64

In [21]:
ride_ids.sample(2)

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_total_cost
27612,2494554e7702a28816717bc0cbf7d57b,0ee71a1ea6aa8485f25a632c7045b0c1,974,290,0,5.508915
51749,402ce9ac2db62fcb5fe29ba1d40566c7,7c67f7fcdbb57e0c94fda81da7c2b9c8,1911,264,50,8.249111


In [10]:
print('Shape of data:', ride_ids.shape)
print('unique of driver id:', len(ride_ids['driver_id'].unique()))
print('unique of ride id:', len(ride_ids['ride_id'].unique()))
ride_ids.describe()

Shape of data: (193502, 5)
unique of driver id: 937
unique of ride id: 193502


,ride_distance,ride_duration,ride_prime_time
count,193502.000000,193502.000000,193502.000000
mean,6955.218266,858.966099,17.305893
std,8929.444606,571.375818,30.825800
min,-2.000000,2.000000,0.000000
25%,2459.000000,491.000000,0.000000
50%,4015.000000,727.000000,0.000000
75%,7193.000000,1069.000000,25.000000
max,724679.000000,28204.000000,500.000000


In [13]:
basefare = 2.00
costpermile = 1.15
costpermin = 0.22
servicefee = 1.75
minfare = 5.00
maxfare = 400.00
ride_ids['ride_total_cost'] = ((basefare + (costpermin * (ride_ids['ride_duration']/60)) + (costpermile * (ride_ids['ride_distance'] * 0.000621))) * (1 + ride_ids['ride_prime_time']/100)) + servicefee
ride_ids['ride_total_cost'] = np.where(ride_ids['ride_total_cost'] < minfare, minfare, ride_ids['ride_total_cost'])
ride_ids['ride_total_cost'] = np.where(ride_ids['ride_total_cost'] > maxfare, maxfare, ride_ids['ride_total_cost'])
ride_ids.head(3)

# Cost of Lyft ride: (basefare + cost per mile * ride_distance + cost per minute * ride_duration)
# * (1 + ride_prime_time/100) + service fee

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_total_cost
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,8.488488
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,9.117306
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,8.191174


In [19]:
ride_events = ride_timestamps.pivot(index = 'ride_id', columns = 'event', values = 'timestamp')
ride_events.head(3)

event,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at
ride_id,,,,,
00003037a262d9ee40e61b5c0718f7f0,2016-06-13 09:39:51,2016-06-13 09:44:31,2016-06-13 10:03:05,2016-06-13 09:44:33,2016-06-13 09:39:19
00005eae40882760d675da5effb89ae3,2016-05-14 05:23:25,2016-05-14 05:26:17,2016-05-14 05:34:17,2016-05-14 05:26:18,2016-05-14 05:23:21
000061d42cf29f73b591041d9a1b2973,2016-05-16 15:43:14,2016-05-16 15:47:29,2016-05-16 15:54:18,2016-05-16 15:47:32,2016-05-16 15:43:09


In [58]:
ride_data = pd.merge(ride_ids, ride_events, on='ride_id')

In [59]:
# convert ride distance from meters to kilometers
ride_data["ride_distance"] = ride_data["ride_distance"]/1000
# convert ride duration from seconds to minutes 
ride_data["ride_duration"] = ride_data["ride_duration"]/60
#Create Features
ride_data['ride_accept_response_time'] = (pd.to_datetime(ride_data['accepted_at']) - pd.to_datetime(ride_data['requested_at'])).astype('timedelta64[s]')/60
# It is the time a driver takes to reach the pickup location once a ride is accepted in minutes.
ride_data['ride_arrival_time'] = (pd.to_datetime(ride_data['arrived_at']) - pd.to_datetime(ride_data['accepted_at'])).astype('timedelta64[s]')/60
# It is the time a driver waits for the rider once the driver reaches the pickup location in minutes.
ride_data['ride_wait_time'] = (pd.to_datetime(ride_data['picked_up_at']) - pd.to_datetime(ride_data['arrived_at'])).astype('timedelta64[s]')/60
# divide ride distance to 3 categories (Short ride, medium ride, long ride) dependant on ride_distance column (KM)
ride_data["ride_length"] = np.where(ride_data["ride_distance"] > ride_data['ride_distance'].quantile(0.33), (np.where(ride_data["ride_distance"] < ride_data['ride_distance'].quantile(0.66), "med_ride", "long_ride")), "short_ride")
# divide ride time to 3 categories (Morning ride, evening ride, night ride) dependant on accepted_at column (time)
ride_data["ride_time"] = np.where(pd.to_datetime(ride_data['accepted_at']).dt.hour > 6, (np.where(pd.to_datetime(ride_data['accepted_at']).dt.hour <= 15, "morning_ride", (np.where(pd.to_datetime(ride_data['accepted_at']).dt.hour <= 21, "evening_ride", "night_ride")))), "night_ride")
# Convert accepted_at column to date
ride_data['ride_date'] = pd.to_datetime(ride_data['accepted_at']).dt.date
# Print first 3 rows in ride data
ride_data.head(5)

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,ride_total_cost,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at,ride_accept_response_time,ride_arrival_time,ride_wait_time,ride_length,ride_time,ride_date
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1.811,5.450000,50,8.488488,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:22:07,2016-04-23 02:16:40,2016-04-23 02:13:50,0.416667,2.350000,0.066667,short_ride,night_ride,2016-04-23
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3.362,13.483333,0,9.117306,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:17:30,2016-03-29 19:04:01,2016-03-29 19:00:49,0.050000,3.083333,0.066667,med_ride,evening_ride,2016-03-29
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3.282,9.533333,0,8.191174,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:11:07,2016-06-21 12:01:35,2016-06-21 11:56:31,0.133333,4.883333,0.050000,med_ride,morning_ride,2016-06-21
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65.283,55.633333,25,77.826485,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 10:13:58,2016-05-19 09:18:20,2016-05-19 09:15:29,0.066667,2.783333,0.000000,long_ride,morning_ride,2016-05-19
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4.115,13.716667,100,17.662788,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:20:45,2016-04-20 22:07:02,2016-04-20 22:05:30,0.033333,1.516667,-0.016667,med_ride,night_ride,2016-04-20


In [61]:
driver_perday_ridecount = ride_data.pivot_table(index = 'driver_id', columns = 'ride_date', aggfunc = 'size').fillna(0)
driver_perday_ridecount.head(5)

ride_date,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,...,2016-06-17,2016-06-18,2016-06-19,2016-06-20,2016-06-21,2016-06-22,2016-06-23,2016-06-24,2016-06-25,2016-06-26
driver_id,,,,,,,,,,,,,,,,,,,,,
002be0ffdc997bd5c50703158b7c2491,0.0,6.0,0.0,8.0,7.0,5.0,0.0,0.0,7.0,0.0,...,1.0,3.0,0.0,2.0,9.0,3.0,5.0,0.0,0.0,0.0
007f0389f9c7b03ef97098422f902e62,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
011e5c5dfc5c2c92501b8b24d47509bc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0152a2f305e71d26cc964f8d4411add9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,6.0,12.0,10.0,5.0,6.0,3.0
01674381af7edd264113d4e6ed55ecda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,9.0,10.0,13.0,13.0,9.0,0.0,0.0


In [63]:
driver_info = pd.DataFrame()
# Calc. driver total ride count
driver_info['total_ride_count'] = ride_data.pivot_table(index=['driver_id'], aggfunc='size')
# Calc. driver total ride distance
driver_info['total_distance']  = ride_ids.groupby('driver_id')['ride_distance'].sum()
# Calc. driver total ride duration
driver_info['total_duration']  = ride_ids.groupby('driver_id')['ride_duration'].sum()
# Calc. driver lifetime value (total revenue)
driver_info['lifetime_value'] = ride_ids.groupby('driver_id')['ride_total_cost'].sum()
# Calc. mean driver count ride per day
driver_info['perday_ridecount_mean'] = driver_perday_ridecount.mean(axis=1)
# Calc. mean of ride accept responce time for every driver
driver_info['accept_response_time_mean'] = (ride_data.groupby(ride_data['driver_id']).aggregate({'ride_accept_response_time': 'mean'})).iloc[:,0]
# Calc. mean of ride arrival time for every driver
driver_info['arrival_time_mean'] = (ride_data.groupby(ride_data['driver_id']).aggregate({'ride_arrival_time': 'mean'})).iloc[:,0]
# Calc. mean of ride wait time for every driver
driver_info['wait_time_mean'] = (ride_data.groupby(ride_data['driver_id']).aggregate({'ride_wait_time': 'mean'})).iloc[:,0]
# count total no. of short/medium/long evening rides
driver_info = pd.merge(driver_info, ride_data.pivot_table(index = 'driver_id', columns = 'ride_length', aggfunc='size') ,on='driver_id')
# count total no. of day/night/night evening rides
driver_info = pd.merge(driver_info, ride_data.pivot_table(index = 'driver_id', columns = 'ride_time', aggfunc='size') ,on='driver_id')

# fill nan value with 0
driver_info = driver_info.fillna(0).reset_index()
driver_info.head(5)

,driver_id,total_ride_count,total_distance,total_duration,lifetime_value,perday_ridecount_mean,accept_response_time_mean,arrival_time_mean,wait_time_mean,long_ride,med_ride,short_ride,evening_ride,morning_ride,night_ride
0,002be0ffdc997bd5c50703158b7c2491,277,1740287,221238,3560.056648,3.043956,0.140132,2.883153,0.087184,79,103,95,103.0,85.0,89.0
1,007f0389f9c7b03ef97098422f902e62,31,117531,20497,321.434698,0.340659,0.197312,2.952688,0.081183,5,15,11,24.0,6.0,1.0
2,011e5c5dfc5c2c92501b8b24d47509bc,34,269653,29205,482.299206,0.373626,0.121569,3.720588,0.068137,11,11,12,24.0,8.0,2.0
3,0152a2f305e71d26cc964f8d4411add9,191,1471239,174521,2610.189071,2.098901,0.236649,3.908813,0.091885,99,51,41,17.0,67.0,107.0
4,01674381af7edd264113d4e6ed55ecda,375,3123644,357443,5379.644866,4.120879,0.208800,3.608133,0.136800,157,111,107,0.0,350.0,25.0


In [64]:
# Calc how long do the driver typically continue driving with Lyft
onboard_data = pd.merge(ride_data.groupby('driver_id').agg({'accepted_at': max}), driver_ids , on='driver_id')
# Calc. driver_duration the long of how driver continue driving with lyft par days
onboard_data['driver_duration'] = (pd.to_datetime(onboard_data['accepted_at']) - pd.to_datetime(onboard_data['driver_onboard_date'])).astype('timedelta64[D]')
onboard_data = onboard_data[['driver_id', 'driver_duration']]
onboard_data

,driver_id,driver_duration
0,002be0ffdc997bd5c50703158b7c2491,86.0
1,007f0389f9c7b03ef97098422f902e62,85.0
2,011e5c5dfc5c2c92501b8b24d47509bc,68.0
3,0152a2f305e71d26cc964f8d4411add9,64.0
4,01674381af7edd264113d4e6ed55ecda,56.0
...,...,...
832,ff419a3476e21e269e340b5f1f05414e,61.0
833,ff714a67ba8c6a108261cd81e3b77f3a,81.0
834,fff482c704d36a1afe8b8978d5486283,34.0
835,fffecccc49436c5389075b13209f0dfa,51.0


In [65]:
driver_info = pd.merge(driver_info, onboard_data, on = 'driver_id', how = 'inner')
driver_info

,driver_id,total_ride_count,total_distance,total_duration,lifetime_value,perday_ridecount_mean,accept_response_time_mean,arrival_time_mean,wait_time_mean,long_ride,med_ride,short_ride,evening_ride,morning_ride,night_ride,driver_duration
0,002be0ffdc997bd5c50703158b7c2491,277,1740287,221238,3560.056648,3.043956,0.140132,2.883153,0.087184,79,103,95,103.0,85.0,89.0,86.0
1,007f0389f9c7b03ef97098422f902e62,31,117531,20497,321.434698,0.340659,0.197312,2.952688,0.081183,5,15,11,24.0,6.0,1.0,85.0
2,011e5c5dfc5c2c92501b8b24d47509bc,34,269653,29205,482.299206,0.373626,0.121569,3.720588,0.068137,11,11,12,24.0,8.0,2.0,68.0
3,0152a2f305e71d26cc964f8d4411add9,191,1471239,174521,2610.189071,2.098901,0.236649,3.908813,0.091885,99,51,41,17.0,67.0,107.0,64.0
4,01674381af7edd264113d4e6ed55ecda,375,3123644,357443,5379.644866,4.120879,0.208800,3.608133,0.136800,157,111,107,0.0,350.0,25.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,ff419a3476e21e269e340b5f1f05414e,242,2002400,246308,3537.441127,2.659341,0.218939,4.651997,0.114738,98,56,88,56.0,161.0,25.0,61.0
833,ff714a67ba8c6a108261cd81e3b77f3a,485,2834765,442492,6285.948397,5.329670,0.177663,2.976426,0.048144,161,155,169,164.0,308.0,13.0,81.0
834,fff482c704d36a1afe8b8978d5486283,35,174394,27693,380.579655,0.384615,0.303333,3.835238,0.034762,11,7,17,1.0,34.0,0.0,34.0
835,fffecccc49436c5389075b13209f0dfa,406,2924913,353974,6050.600054,4.461538,0.176929,3.572619,0.191051,137,141,128,135.0,153.0,118.0,51.0


In [66]:
driver_info.to_csv('driver_info.csv')